In [61]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

## Scrape stock new

In [62]:
def get_soup(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

In [63]:
hoon_stock=get_soup('https://www.hooninside.com/news-feed/316062/view/')

In [64]:
ignore_word=['อีเมล์\:','Copyrights','สามารถติดตามหน้าเพจของ หุ้นอินไซด์']
ignore_word_pattern='|'.join(ignore_word)

In [65]:
texts=[]
for p in hoon_stock.find_all('p'):
    # ข้อความของ element ที่มี tag p
    context=p.text
    if len( context)>50  and re.search(ignore_word_pattern,context)==None:
        # print(p)
        texts.append(context)

# Summarizing using Openai API

In [66]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [67]:
def get_completion_and_token_count(prompt, 
                                   model="gpt-3.5-turbo", 
                                   temperature=0, 
                                   max_tokens=300):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    content = response.choices[0].message["content"]
    
    token_dict = {
'prompt_tokens':response['usage']['prompt_tokens'],
'completion_tokens':response['usage']['completion_tokens'],
'total_tokens':response['usage']['total_tokens'],
    }

    return content, token_dict

Need to analyze the amount of words that suitable for summary

In [68]:
import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")
len(encoding.encode('ในไตรมาส 3 ปี 2566 รายได้จากการดำเนินงานเพิ่มขึ้น 9.0% เป็น 43,344 ล้านบาท โดยรายได้ดอกเบี้ยสุทธิเพิ่มขึ้น 13.8% และรายได้ค่าธรรมเนียมลดลง 9.2% รายได้จากการลงทุนเพิ่มขึ้นเป็น 1,141 ล้านบาท'))

176

## Summary new each paragraph

In [69]:
paragraph_sum={}
for i,new in enumerate(texts):
    print('original:',new)
    encoding = tiktoken.get_encoding("cl100k_base")
    text_token=len(encoding.encode(new))
    #summary only paragraph more than 150 token
    if text_token>150:
        prompt = f"""
        Your task is to generate a short summary in Thai of stock news. 
        Summarize in Thai the review below in at most 20 words.
        Summarize: ```{new}```
        """

        response, token_dict= get_completion_and_token_count(prompt)
        print('Summary:',response, token_dict)

        paragraph_sum[i]=response
    else:
        paragraph_sum[i]=new

original: สำนักข่าวหุ้นอินไซด์(20 ตุลาคม 2566)----บริษัท เอสซีบี เอกซ์ จำกัด (มหาชน) มีกำไรสุทธิในไตรมาส 3 ของปี 2566 จากบริษัทในกลุ่มทั้งหมดจำนวน 9,663 ล้านบาท ลดลง 6.3% จากช่วงเดียวกันของปีก่อน โดยมีการตั้งสำรองพิเศษเพิ่มอีก จำนวน 1,500 ล้านบาท เพื่อรองรับความเสี่ยงในอนาคตจากสถานการณ์ภูมิรัฐศาสตร์ที่มีความไม่แน่นอนสูงขึ้น สำหรับเก้าเดือนแรกของปี บริษัทฯ มีกำไรสุทธิจำนวน 32,527 ล้านบาท เพิ่มขึ้น 7.0% ในส่วนของธนาคารไทยพาณิชย์ซึ่งเป็นบริษัทหลักในกลุ่ม มีกำไรสุทธิในไตรมาส 3 จำนวน 11,782 ล้านบาท ลดลง 6.3% และมีกำไรสุทธิสำหรับเก้าเดือนแรกของปีจำนวน 36,627 ล้านบาท เพิ่มขึ้น 21.2%


Summary: บริษัท เอสซีบี เอกซ์ จำกัด (มหาชน) มีกำไรสุทธิในไตรมาส 3 ของปี 2566 ลดลง 6.3% และมีการตั้งสำรองพิเศษเพิ่มอีก 1,500 ล้านบาท เพื่อรองรับความเสี่ยงในอนาคต ส่วนธนาคารไทยพาณิชย์มีกำไรสุทธิในไตรมาส 3 ลดลง 6.3% และมีกำไรสุทธิในเก้าเดือนแรกของปีเพิ่มขึ้น 21.2% {'prompt_tokens': 592, 'completion_tokens': 244, 'total_tokens': 836}
original: ในไตรมาส 3 ของปี 2566 รายได้จากการดำเนินงานมีจำนวน 43,344 ล้านบาทเพิ่มขึ้น 9.0% โดยแบ่งเป็นรายได้ดอกเบี้ยสุทธิจำนวน 31,536 ล้านบาท เพิ่มขึ้น 13.8% จากการเติบโตของสินเชื่อ และการขยายตัวของส่วนต่างดอกเบี้ยสุทธิ (NIM) รายได้ค่าธรรมเนียมและอื่น ๆ จำนวน 10,667 ล้านบาท ลดลง 9.2% และรายได้จากการลงทุนและการค้าเพิ่มขึ้นเกือบสามเท่าตัวจากปีที่แล้วมาเป็น 1,141 ล้านบาท
Summary: ในไตรมาส 3 ปี 2566 รายได้จากการดำเนินงานเพิ่มขึ้น 9.0% เป็น 43,344 ล้านบาท โดยรายได้ดอกเบี้ยสุทธิเพิ่มขึ้น 13.8% และรายได้ค่าธรรมเนียมลดลง 9.2% รายได้จากการลงทุนเพิ่มขึ้นเป็น 1,141 ล้านบาท {'prompt_tokens': 375, 'completion_tokens': 176, 'total_tokens': 551}
original: ค่าใช้จ่ายในการดำเนิ

In [70]:
paragraph_sum

{0: 'บริษัท เอสซีบี เอกซ์ จำกัด (มหาชน) มีกำไรสุทธิในไตรมาส 3 ของปี 2566 ลดลง 6.3% และมีการตั้งสำรองพิเศษเพิ่มอีก 1,500 ล้านบาท เพื่อรองรับความเสี่ยงในอนาคต ส่วนธนาคารไทยพาณิชย์มีกำไรสุทธิในไตรมาส 3 ลดลง 6.3% และมีกำไรสุทธิในเก้าเดือนแรกของปีเพิ่มขึ้น 21.2%',
 1: 'ในไตรมาส 3 ปี 2566 รายได้จากการดำเนินงานเพิ่มขึ้น 9.0% เป็น 43,344 ล้านบาท โดยรายได้ดอกเบี้ยสุทธิเพิ่มขึ้น 13.8% และรายได้ค่าธรรมเนียมลดลง 9.2% รายได้จากการลงทุนเพิ่มขึ้นเป็น 1,141 ล้านบาท',
 2: 'ค่าใช้จ่ายในการดำเนินงานมีจำนวน 18,490 ล้านบาท คิดเป็นอัตราส่วนค่าใช้จ่ายต่อรายได้ที่ 42.7% ซึ่งสะท้อนถึงการบริหารต้นทุนที่มีประสิทธิภาพอย่างต่อเนื่อง',
 3: 'บริษัทฯ ได้เพิ่มเงินสำรองพิเศษ 1,500 ล้านบาท เพื่อเตรียมความไม่แน่นอนในสถานการณ์ภูมิรัฐศาสตร์ ทำให้เงินสำรองในไตรมาสนี้เป็น 12,245 ล้านบาท อัตราส่วนผลขาดทุนด้านเครดิตคาดว่าจะเป็น 2.01% และอัตราส่วนค่าเผื่อหนี้สงสัยคงอยู่ในระดับสูงที่ 167.2%',
 4: 'คุณภาพสินเชื่อดีขึ้นเล็กน้อยในเดือนกันยายน 2566 ที่ 3.30% และเงินกองทุนรวมยังแข็งแกร่งที่ 18.7%.',
 5: 'นายอาทิตย์ นันทวิทยา ประธานเจ

In [71]:
summary_all=[]
for i,p in paragraph_sum.items():
    summary_list+=p
    summary_new=' '.join(summary_list)
    text_token=len(encoding.encode(summary_new))
    # commulative the token of paragraph summary >1000 then summary
    if text_token>1000:
        print(summary_list)
        print(text_token)
        prompt = f"""
        Your task is to generate a short summary in Thai of stock news. 
        Summarize in Thai the review below in 20 words.
        Summarize: ```{summary_new}```
        """

        response, token_dict= get_completion_and_token_count(prompt)
        print('Summary:',response, token_dict)
        summary_all.append(response)
        summary_list=''

ค่าใช้จ่ายในการดำเนินงานมีจำนวน 18,490 ล้านบาท คิดเป็นอัตราส่วนค่าใช้จ่ายต่อรายได้ที่ 42.7% ซึ่งสะท้อนถึงการบริหารต้นทุนที่มีประสิทธิภาพอย่างต่อเนื่องบริษัทฯ ได้เพิ่มเงินสำรองพิเศษ 1,500 ล้านบาท เพื่อเตรียมความไม่แน่นอนในสถานการณ์ภูมิรัฐศาสตร์ ทำให้เงินสำรองในไตรมาสนี้เป็น 12,245 ล้านบาท อัตราส่วนผลขาดทุนด้านเครดิตคาดว่าจะเป็น 2.01% และอัตราส่วนค่าเผื่อหนี้สงสัยคงอยู่ในระดับสูงที่ 167.2%บริษัท เอสซีบี เอกซ์ จำกัด (มหาชน) มีกำไรสุทธิในไตรมาส 3 ของปี 2566 ลดลง 6.3% และมีการตั้งสำรองพิเศษเพิ่มอีก 1,500 ล้านบาท เพื่อรองรับความเสี่ยงในอนาคต ส่วนธนาคารไทยพาณิชย์มีกำไรสุทธิในไตรมาส 3 ลดลง 6.3% และมีกำไรสุทธิในเก้าเดือนแรกของปีเพิ่มขึ้น 21.2%
1229
Summary: ค่าใช้จ่ายในการดำเนินงานมีจำนวน 18,490 ล้านบาท คิดเป็นอัตราส่วนค่าใช้จ่ายต่อรายได้ที่ 42.7% ซึ่งสะท้อนถึงการบริหารรต้นทุนที่มีประสิทธิภาพอย่างต่อเนื่อง บริษัทได้เพิ่มเงินสำรองพิเศษ 1,500 ล้านบาทเพื่อเตรียมความไม่แน่นอนในสถานการณ์ภูมิรัฐศาสตร์ ทำให้เงินสำรองในไตรมาสนี้เป็น 12,245 ล้านบาท อัตราส่วนผลขาดทุนด้านเครดิตคาดว่าจะเป {'prompt_tokens':

In [72]:
print(summary_list)
print(text_token)
prompt = f"""
        Your task is to generate a short summary in Thai of stock news. 
        Summarize in Thai the review below in 20 words.
        Summarize: ```{summary_new}```
        """

response, token_dict= get_completion_and_token_count(prompt)
print('Summary:',response, token_dict)
summary_all.append(response)

บริษัทฯ จ่ายเงินปันผลระหว่างกาลปี 2566 ที่ 2.5 บาทต่อหุ้น ส่งสัญญาณให้เห็นถึงความมุ่งมั่นในการเพิ่มมูลค่าผู้ถือหุ้นและสถานะการเงินที่แข็งแกร่ง"พีทีจี เอ็นเนอยี" จับมือ "เอส 63 โปรเจค" ระเบิดความมันส์ "พีที สงขลา กรังด์ปรีซ์ 2023" สุดยิ่งใหญ่!"ชูวิทย์" NER คว้า สุดยอด CEO รุ่นใหญ่ ปี 2566 ตอกย้ำความเชื่อมั่นในการดำเนินงาน พร้อมขับเคลื่อนธุรกิจให้เติบโตอย่างยMCA เป็นผู้ให้บริการทางกลยุทธ์ และการจัดกิจกรรมทางการตลาดเพื่อสนับสนุนการขายที่ครบวงจร 
840
Summary: บริษัทฯ จ่ายเงินปันผลระหว่างกาลปี 2566 ที่ 2.5 บาทต่อหุ้น ส่งสัญญาณให้เห็นถึงความมั่นในการเพิ่มมูลค่าผู้ถือหุ้นและสถานะการเงินที่แข็งแกร่ง "พีทีจี เอ็นเนอร์ยี" จับมือ "เอส 63 โปรเจค" ระเบิดความมั่นส์ "พีที สงขลา กรังด์ปรีซ์ 2023" สุดยิ่งใหญ่! "ชูวิทย์" NER คว่าสุดยอด CEO รุ่นใหญ่ ปี 2566 ตอกย้ำความเชื่อมั่นในการดำเนินงาน พร้อมขับเคล {'prompt_tokens': 886, 'completion_tokens': 300, 'total_tokens': 1186}


## Result_summary_new

In [74]:
result_summary=' '.join(summary_all)
prompt = f"""
        Your task is to generate a short summary in Thai of stock news. 
        Summarize in Thai the review below in 20 words.
        Summarize: ```{result_summary}```
        """

response, token_dict= get_completion_and_token_count(prompt,max_tokens=500)
print('Summary:',response, token_dict)

Summary: บริษัทเอสซีบี เอกซ์ จำกัด (มหาชน) ได้เพิ่มเงินสำรองพิเศษ 1,500 ล้านบาทเพื่อเตรียมความไม่แน่นอนในสถานการณ์ภูมิรัฐศาสตร์ รายได้จากการดำเนินงานเพิ่มขึ้น 9.0% เป็น 43,344 ล้านบาท และรายได้ค่าธรรมเนียมลดลง 9.2% รายได้จากการลงทุนเพิ่มขึ้นเป็น 1,141 ล้านบาท คุณภาพสินเชื่อดีขึ้นเล็กน้อยในเดือนกันยายน 2566 ที่ 3.30% และเงินกองทุนรวมยังแข็งแกร่งแก้งแกร่งที่ 18.7%. บริษัทฯ จ่ายเงินปันผลระหว่างกาลปี 2566 ที่ 2.5 บาทต่อหุ้น. {'prompt_tokens': 1248, 'completion_tokens': 382, 'total_tokens': 1630}
